In [2]:
!pip install ultralytics opencv-python matplotlib pyyaml shapely
!pip install torch torchvision

In [3]:
import os
import cv2
import numpy as np
from ultralytics import YOLO
import matplotlib.pyplot as plt
from pathlib import Path
import yaml
from shapely.geometry import Polygon
import matplotlib.patches as patches

In [10]:
class RulerDetector:
    def __init__(self, model_path=None):
        """
        Inicializa o detector de régua para seu dataset específico

        Args:
            model_path (str): Caminho para o modelo treinado. Se None, usa modelo base YOLOv11
        """
        if model_path and os.path.exists(model_path):
            self.model = YOLO(model_path)
            print(f"✅ Modelo carregado de: {model_path}")
        else:
            # Usa modelo de segmentação para aproveitar coordenadas de polígono
            self.model = YOLO('yolo11n-seg.pt')
            print("🔄 Modelo base YOLOv11n-seg carregado para treinamento")

        # Configurações específicas do seu dataset
        self.class_names = ['0', 'Penggaris', 'Penghapus', 'Pulpen']
        self.ruler_class_id = 1  # 'Penggaris' é a classe 1 (régua)

        print(f"🎯 Classes do dataset: {self.class_names}")
        print(f"📏 Classe da régua: {self.class_names[self.ruler_class_id]} (ID: {self.ruler_class_id})")

    def verify_dataset_structure(self, dataset_path):
        """
        Verifica e exibe a estrutura do dataset
        """
        dataset_path = Path(dataset_path)
        print(f"\n📁 Verificando estrutura do dataset em: {dataset_path}")

        # Verifica pastas principais
        folders_found = {}
        for folder in ['train', 'valid', 'test']:
            folder_path = dataset_path / folder
            if folder_path.exists():
                images_path = folder_path / 'images'
                labels_path = folder_path / 'labels'

                img_count = len(list(images_path.glob('*'))) if images_path.exists() else 0
                lbl_count = len(list(labels_path.glob('*.txt'))) if labels_path.exists() else 0

                folders_found[folder] = {
                    'images': img_count,
                    'labels': lbl_count,
                    'images_exist': images_path.exists(),
                    'labels_exist': labels_path.exists()
                }

                print(f"  📂 {folder}/: {img_count} imagens, {lbl_count} labels")

        return folders_found

    def analyze_annotations(self, dataset_path, sample_size=5):
        """
        Analisa anotações para entender o formato
        """
        dataset_path = Path(dataset_path)
        print(f"\n🔍 Analisando formato das anotações...")

        # Procura arquivos de anotação
        label_files = []
        for folder in ['train', 'valid']:
            labels_path = dataset_path / folder / 'labels'
            if labels_path.exists():
                label_files.extend(list(labels_path.glob('*.txt'))[:sample_size])

        if not label_files:
            print("❌ Nenhum arquivo de anotação encontrado!")
            return

        annotation_info = {
            'total_files_analyzed': 0,
            'classes_found': set(),
            'ruler_annotations': 0,
            'sample_annotations': []
        }

        for label_file in label_files[:sample_size]:
            if not label_file.exists():
                continue

            annotation_info['total_files_analyzed'] += 1

            with open(label_file, 'r') as f:
                lines = f.readlines()

            for line in lines:
                if line.strip():
                    parts = list(map(float, line.strip().split()))
                    class_id = int(parts[0])
                    coords = parts[1:]

                    annotation_info['classes_found'].add(class_id)

                    if class_id == self.ruler_class_id:
                        annotation_info['ruler_annotations'] += 1

                        if len(annotation_info['sample_annotations']) < 2:
                            annotation_info['sample_annotations'].append({
                                'file': label_file.name,
                                'class_id': class_id,
                                'num_points': len(coords) // 2,
                                'coords_sample': coords[:8]  # Primeiros 4 pontos
                            })

        # Exibe resultados da análise
        print(f"📊 Arquivos analisados: {annotation_info['total_files_analyzed']}")
        print(f"🏷️ Classes encontradas: {sorted(annotation_info['classes_found'])}")
        print(f"📏 Anotações de régua encontradas: {annotation_info['ruler_annotations']}")

        for sample in annotation_info['sample_annotations']:
            print(f"  📝 {sample['file']}: Classe {sample['class_id']}, {sample['num_points']} pontos")

        return annotation_info

    def setup_dataset(self, dataset_path):
        """
        Configura o dataset para treinamento
        """
        self.dataset_path = Path(dataset_path)

        # Verifica estrutura
        self.verify_dataset_structure(dataset_path)

        # Analisa anotações
        self.analyze_annotations(dataset_path)

        # Verifica/cria data.yaml
        config_path = self.dataset_path / "data.yaml"
        if config_path.exists():
            print(f"✅ Arquivo data.yaml encontrado")
            # Verifica se está correto
            with open(config_path, 'r') as f:
                config = yaml.safe_load(f)
            print(f"   - Classes: {config.get('names', 'não encontradas')}")
        else:
            print("📝 Criando arquivo data.yaml...")
            self.create_dataset_config(config_path)

        return str(config_path)

    def create_dataset_config(self, config_path):
        """
        Cria arquivo de configuração do dataset baseado na sua estrutura
        """
        config = {
            'path': str(self.dataset_path.absolute()),
            'train': 'train/images',
            'val': 'valid/images',
            'test': 'test/images',
            'nc': 4,
            'names': ['0', 'Penggaris', 'Penghapus', 'Pulpen']
        }

        with open(config_path, 'w') as f:
            yaml.dump(config, f, default_flow_style=False)

        print(f"✅ Arquivo data.yaml criado: {config_path}")

    def train_model(self, dataset_path, epochs=100, img_size=640, batch_size=8, patience=20):
        """
        Treina o modelo de segmentação
        """
        print("🚀 Iniciando treinamento do modelo de segmentação...")

        # Configura dataset
        config_file = self.setup_dataset(dataset_path)

        try:
            # Treina o modelo
            results = self.model.train(
                data=config_file,
                epochs=epochs,
                imgsz=img_size,
                batch=batch_size,
                name='ruler_segmentation',
                save=True,
                plots=True,
                patience=patience,
                device='cpu'  # Usa GPU se disponível
            )

            model_path = "runs/segment/ruler_segmentation/weights/best.pt"
            print(f"✅ Treinamento concluído!")
            print(f"📁 Modelo salvo em: {model_path}")

            return results

        except Exception as e:
            print(f"❌ Erro durante treinamento: {e}")
            print("💡 Dicas para resolver:")
            print("  - Verifique se as imagens e labels estão nas pastas corretas")
            print("  - Reduza o batch_size se houver erro de memória")
            print("  - Verifique se o dataset está no formato correto")
            raise

    def load_trained_model(self, model_path):
        """
        Carrega modelo já treinado
        """
        if not os.path.exists(model_path):
            print(f"❌ Modelo não encontrado: {model_path}")
            return False

        self.model = YOLO(model_path)
        print(f"✅ Modelo carregado: {model_path}")
        return True

    def calculate_polygon_area_shoelace(self, points):
        """
        Calcula área de polígono usando fórmula Shoelace
        Mais precisa que usar apenas bounding box
        """
        if len(points) < 6:  # Precisa de pelo menos 3 pontos (6 coordenadas)
            return 0

        # Organiza pontos em pares (x, y)
        coords = []
        for i in range(0, len(points), 2):
            if i + 1 < len(points):
                coords.append([points[i], points[i + 1]])

        if len(coords) < 3:
            return 0

        # Fórmula Shoelace
        area = 0
        n = len(coords)
        for i in range(n):
            j = (i + 1) % n
            area += coords[i][0] * coords[j][1]
            area -= coords[j][0] * coords[i][1]

        return abs(area) / 2

    def detect_ruler(self, image_path, conf_threshold=0.3, save_crops=False):
        """
        Detecta régua na imagem e calcula área real do polígono
        """
        if not os.path.exists(image_path):
            return {"error": f"Imagem não encontrada: {image_path}"}

        print(f"🔍 Analisando imagem: {Path(image_path).name}")

        # Faz predição
        results = self.model(image_path, conf=conf_threshold, save_crop=save_crops)

        # Carrega imagem para obter dimensões
        image = cv2.imread(image_path)
        if image is None:
            return {"error": f"Não foi possível carregar a imagem"}

        img_height, img_width = image.shape[:2]

        detection_info = {
            "image_path": image_path,
            "image_dimensions": {"width": img_width, "height": img_height},
            "ruler_detected": False,
            "ruler_detections": [],
            "all_detections": [],
            "total_ruler_area_pixels": 0,
            "confidence_threshold": conf_threshold
        }

        # Processa resultados
        for result in results:
            if result.boxes is not None:
                for i, box in enumerate(result.boxes):
                    class_id = int(box.cls[0].cpu().numpy())
                    confidence = float(box.conf[0].cpu().numpy())
                    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()

                    # Informações básicas da detecção
                    detection = {
                        "class_id": class_id,
                        "class_name": self.class_names[class_id],
                        "confidence": confidence,
                        "bbox": [int(x1), int(y1), int(x2), int(y2)],
                        "bbox_area_pixels": int((x2 - x1) * (y2 - y1))
                    }

                    # Tenta obter máscara de segmentação
                    if (hasattr(result, 'masks') and
                        result.masks is not None and
                        i < len(result.masks.xy)):

                        # Coordenadas da máscara (já em pixels)
                        mask_coords = result.masks.xy[i]
                        if len(mask_coords) > 0:
                            # Converte para lista de coordenadas
                            coords_flat = mask_coords.flatten().tolist()
                            detection["segmentation_coords"] = coords_flat
                            detection["num_points"] = len(coords_flat) // 2

                            # Calcula área real do polígono
                            polygon_area = self.calculate_polygon_area_shoelace(coords_flat)
                            detection["polygon_area_pixels"] = int(polygon_area)

                            print(f"  🎯 {detection['class_name']}: {detection['num_points']} pontos, área: {polygon_area:.0f} px")

                    detection_info["all_detections"].append(detection)

                    # Se é régua, adiciona às detecções de régua
                    if class_id == self.ruler_class_id:
                        detection_info["ruler_detected"] = True
                        detection_info["ruler_detections"].append(detection)

                        # Soma área (usa polígono se disponível, senão bbox)
                        area = detection.get("polygon_area_pixels", detection["bbox_area_pixels"])
                        detection_info["total_ruler_area_pixels"] += area

        # Resumo final
        if detection_info["ruler_detected"]:
            print(f"✅ {len(detection_info['ruler_detections'])} régua(s) detectada(s)")
            print(f"📐 Área total: {detection_info['total_ruler_area_pixels']} pixels")
        else:
            print("❌ Nenhuma régua detectada")

        print(f"📊 Total de objetos: {len(detection_info['all_detections'])}")

        return detection_info

    def visualize_detection(self, image_path, detection_info, save_path=None, show_all_objects=True):
        """
        Visualiza resultados com polígonos precisos
        """
        image = cv2.imread(image_path)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        fig, ax = plt.subplots(1, 1, figsize=(16, 12))
        ax.imshow(image_rgb)

        # Cores para cada classe
        class_colors = {
            0: 'blue',
            1: 'red',      # Régua
            2: 'green',    # Borracha
            3: 'orange'    # Caneta
        }

        # Desenha todas as detecções
        for detection in detection_info["all_detections"]:
            class_id = detection["class_id"]
            is_ruler = (class_id == self.ruler_class_id)
            color = class_colors.get(class_id, 'purple')

            # Desenha bounding box
            x1, y1, x2, y2 = detection["bbox"]
            linewidth = 4 if is_ruler else 2
            linestyle = '-' if is_ruler else '--'

            rect = patches.Rectangle((x1, y1), x2-x1, y2-y1,
                                   linewidth=linewidth, edgecolor=color,
                                   facecolor='none', linestyle=linestyle)
            ax.add_patch(rect)

            # Desenha polígono de segmentação se disponível
            if "segmentation_coords" in detection:
                coords = detection["segmentation_coords"]
                # Converte coordenadas planas para pontos (x,y)
                points = []
                for i in range(0, len(coords), 2):
                    if i + 1 < len(coords):
                        points.append([coords[i], coords[i + 1]])

                if len(points) >= 3:
                    polygon = MplPolygon(points, closed=True, fill=False,
                                       edgecolor=color, linewidth=3, alpha=0.8)
                    ax.add_patch(polygon)

            # Informações do texto
            confidence = detection["confidence"]
            bbox_area = detection["bbox_area_pixels"]
            polygon_area = detection.get("polygon_area_pixels", None)

            # Monta texto informativo
            if is_ruler:
                if polygon_area:
                    text = f'🎯 RÉGUA\nConf: {confidence:.2f}\nÁrea real: {polygon_area:,} px\nBBox: {bbox_area:,} px'
                    text_color = 'white'
                    bg_color = 'red'
                    alpha = 0.9
                else:
                    text = f'📏 RÉGUA\nConf: {confidence:.2f}\nÁrea: {bbox_area:,} px'
                    text_color = 'white'
                    bg_color = 'red'
                    alpha = 0.8
            else:
                if show_all_objects:
                    area_text = f'{polygon_area:,}' if polygon_area else f'{bbox_area:,}'
                    text = f'{detection["class_name"]}\n{confidence:.2f}\n{area_text} px'
                    text_color = 'white'
                    bg_color = color
                    alpha = 0.7
                else:
                    continue  # Pula objetos que não são régua

            # Posiciona texto
            text_y = y1 - 10 if y1 > 50 else y2 + 10
            ax.text(x1, text_y, text,
                   bbox=dict(boxstyle="round,pad=0.5", facecolor=bg_color, alpha=alpha),
                   fontsize=11, color=text_color, weight='bold')

        # Título da imagem
        title = f"📷 {Path(image_path).name}\n"

        if detection_info["ruler_detected"]:
            ruler_count = len(detection_info["ruler_detections"])
            total_area = detection_info["total_ruler_area_pixels"]
            title += f"✅ {ruler_count} Régua(s) Detectada(s) - Área Total: {total_area:,} pixels"
        else:
            title += "❌ Nenhuma Régua Detectada"

        total_objects = len(detection_info["all_detections"])
        title += f"\n📊 Total de Objetos: {total_objects}"

        ax.set_title(title, fontsize=14, weight='bold', pad=20)
        ax.axis('off')

        plt.tight_layout()

        if save_path:
            plt.savefig(save_path, bbox_inches='tight', dpi=200)
            print(f"💾 Resultado salvo em: {save_path}")

        plt.show()

    def batch_process(self, images_folder, output_folder=None):
        """
        Processa múltiplas imagens
        """
        images_folder = Path(images_folder)
        image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.tiff']

        if output_folder:
            output_folder = Path(output_folder)
            output_folder.mkdir(exist_ok=True)

        results = []
        total_ruler_area = 0
        images_with_rulers = 0

        print(f"\n🔄 Processando imagens em: {images_folder}")

        image_files = [f for f in images_folder.iterdir()
                      if f.suffix.lower() in image_extensions]

        for i, img_file in enumerate(image_files, 1):
            print(f"\n📸 [{i}/{len(image_files)}] {img_file.name}")

            # Detecta objetos
            detection_result = self.detect_ruler(str(img_file))
            detection_result["filename"] = img_file.name
            results.append(detection_result)

            # Atualiza estatísticas
            if detection_result["ruler_detected"]:
                images_with_rulers += 1
                total_ruler_area += detection_result["total_ruler_area_pixels"]

            # Salva visualização se solicitado
            if output_folder:
                save_path = output_folder / f"resultado_{img_file.name}"
                self.visualize_detection(str(img_file), detection_result, str(save_path))

        # Resumo final
        print(f"\n📊 === RESUMO DO PROCESSAMENTO ===")
        print(f"🖼️ Imagens processadas: {len(image_files)}")
        print(f"📏 Imagens com réguas: {images_with_rulers}")
        print(f"📐 Área total de réguas: {total_ruler_area:,} pixels")

        if images_with_rulers > 0:
            avg_area = total_ruler_area / images_with_rulers
            print(f"📊 Área média por imagem: {avg_area:.0f} pixels")

        return results


In [11]:
def main():
    print("🎯 === DETECTOR DE RÉGUA - VERSÃO OTIMIZADA ===")
    print("📋 Adaptado especificamente para seu dataset com 4 classes")

    # Configuração
    dataset_path = "/content/drive/MyDrive/MC854/DeteksiPerlengkapanSeklah.v2i.yolov11"  # Substitua pelo caminho real
    test_image = "/content/11T8.JPG"  # Substitua pelo caminho real

    print(f"\n🔧 EXEMPLO DE USO:")
    print(f"1️⃣ TREINAMENTO:")
    print(f"   detector = RulerDetector()")
    print(f"   results = detector.train_model('{dataset_path}', epochs=50)")

    print(f"\n2️⃣ DETECÇÃO:")
    print(f"   detector.load_trained_model('runs/segment/ruler_segmentation/weights/best.pt')")
    print(f"   result = detector.detect_ruler('{test_image}')")
    print(f"   detector.visualize_detection('{test_image}', result)")

    print(f"\n💡 CARACTERÍSTICAS:")
    print(f"   ✅ Detecta todas as 4 classes do seu dataset")
    print(f"   ✅ Calcula área REAL da régua (polígono, não retângulo)")
    print(f"   ✅ Visualização com contornos precisos")
    print(f"   ✅ Processamento em lote de múltiplas imagens")

    # Exemplo prático (descomente para usar)

    # Inicializa detector
    detector = RulerDetector()

    # Treina modelo
    results = detector.train_model(
        dataset_path=dataset_path,
        epochs=50,
        batch_size=8,
        patience=10
    )

    # Testa em imagem
    detector.load_trained_model("runs/segment/ruler_segmentation/weights/best.pt")
    result = detector.detect_ruler(test_image)
    detector.visualize_detection(test_image, result)

    print(f"Régua detectada: {result['ruler_detected']}")
    print(f"Área da régua: {result['total_ruler_area_pixels']} pixels")


In [12]:
if __name__ == "__main__":
    print()

    main()


🎯 === DETECTOR DE RÉGUA - VERSÃO OTIMIZADA ===
📋 Adaptado especificamente para seu dataset com 4 classes

🔧 EXEMPLO DE USO:
1️⃣ TREINAMENTO:
   detector = RulerDetector()
   results = detector.train_model('/content/drive/MyDrive/MC854/DeteksiPerlengkapanSeklah.v2i.yolov11', epochs=50)

2️⃣ DETECÇÃO:
   detector.load_trained_model('runs/segment/ruler_segmentation/weights/best.pt')
   result = detector.detect_ruler('/content/11T8.JPG')
   detector.visualize_detection('/content/11T8.JPG', result)

💡 CARACTERÍSTICAS:
   ✅ Detecta todas as 4 classes do seu dataset
   ✅ Calcula área REAL da régua (polígono, não retângulo)
   ✅ Visualização com contornos precisos
   ✅ Processamento em lote de múltiplas imagens
🔄 Modelo base YOLOv11n-seg carregado para treinamento
🎯 Classes do dataset: ['0', 'Penggaris', 'Penghapus', 'Pulpen']
📏 Classe da régua: Penggaris (ID: 1)
🚀 Iniciando treinamento do modelo de segmentação...

📁 Verificando estrutura do dataset em: /content/drive/MyDrive/MC854/DeteksiPer

train: Scanning /content/drive/MyDrive/MC854/DeteksiPerlengkapanSeklah.v2i.yolov11/train/labels.cache... 313 images, 0 backgrounds, 0 corrupt: 100%|██████████| 313/313 [00:00<?, ?it/s]

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 340, len(boxes) = 444. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.4±0.2 ms, read: 12.3±4.9 MB/s, size: 22.8 KB)


val: Scanning /content/drive/MyDrive/MC854/DeteksiPerlengkapanSeklah.v2i.yolov11/valid/labels.cache... 90 images, 0 backgrounds, 0 corrupt: 100%|██████████| 90/90 [00:00<?, ?it/s]

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 79, len(boxes) = 116. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.


Plotting labels to runs/segment/ruler_segmentation3/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.00125, momentum=0.9) with parameter groups 90 weight(decay=0.0), 101 weight(decay=0.0005), 100 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/segment/ruler_segmentation3
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/40 [00:09<?, ?it/s]

❌ Erro durante treinamento: ERROR ❌ segment dataset incorrectly formatted or not a segment dataset.
This error can occur when incorrectly training a 'segment' model on a 'detect' dataset, i.e. 'yolo train model=yolo11n-seg.pt data=coco8.yaml'.
Verify your dataset is a correctly formatted 'segment' dataset using 'data=coco8-seg.yaml' as an example.
See https://docs.ultralytics.com/datasets/segment/ for help.
💡 Dicas para resolver:
  - Verifique se as imagens e labels estão nas pastas corretas
  - Reduza o batch_size se houver erro de memória
  - Verifique se o dataset está no formato correto


TypeError: ERROR ❌ segment dataset incorrectly formatted or not a segment dataset.
This error can occur when incorrectly training a 'segment' model on a 'detect' dataset, i.e. 'yolo train model=yolo11n-seg.pt data=coco8.yaml'.
Verify your dataset is a correctly formatted 'segment' dataset using 'data=coco8-seg.yaml' as an example.
See https://docs.ultralytics.com/datasets/segment/ for help.